창훈님 전처리 + 재영님 keyword 추출

In [ ]:
pip install contextualized_topic_models

In [ ]:
pip install konlpy

In [3]:
import re
import pandas as pd
import numpy as np
import string
from tensorflow.keras.preprocessing.text import text_to_word_sequence
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from collections import Counter
from contextualized_topic_models.models.ctm import CombinedTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation, bert_embeddings_from_list
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessing
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from konlpy.tag import Okt
from tqdm import tqdm

In [5]:
df = pd.read_csv("/content/drive/MyDrive/my_drama_list/drama_final.csv")
df

,Unnamed: 0,genre,review
0,0,drama,"Well played Reply writers, you psyche-ninjas...."
1,1,drama,"I liked it, but I just wasn't a big fan of it...."
2,2,drama,Slice of Life medical drama with funny moments...
3,3,drama,Is it Worth Investing Time in ” Hospital Playl...
4,4,drama,"% RECOMMENDED HP CASTS, DIRECTOR, WRITER, AND ..."
...,...,...,...
22944,22944,thriller,EXPERT REVIEW\r\n
22945,22945,thriller,The Perfect Conspiracy Series\r\n
22946,22946,thriller,very nice\r\n
22947,22947,thriller,Awesome......\r\n


In [ ]:
drama = df[df['genre']=='roco'].review.to_list()
drama

In [ ]:
sentence = df['review'].to_list()
sentence

In [ ]:
sentence = ",".join(drama)
sentence

In [ ]:
n_gram_range = (3, 3)
stop_words = "english"

# Extract candidate words/phrases
# max_df = 최대빈도수, min_df 최저빈도수, max_features 최종아웃풋 갯수
tfidf = TfidfVectorizer(ngram_range=n_gram_range, stop_words=stop_words, max_features = 10000).fit(drama)
candidates = tfidf.get_feature_names()

print(len(candidates))

In [10]:
def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('V'):
        return 'v'
    elif pos_tag.startswith('N'):
        return 'n'
    elif pos_tag.startswith('J'):
        return 'a'
    elif pos_tag.startswith('R'):
        return 'r'
    else:
        return None

In [ ]:
nltk.download('stopwords')

In [13]:
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))
stop_words.update(('\r', '\n', '\r\n', '\n\r'))

In [14]:
class CustomTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, a):
        a = ''.join(a)
        word_tokens = self.tagger(a)
        
        words = []

        for i in word_tokens:
            text = re.sub('[^a-zA-Z0-9\']','',i).strip()
            text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`…》]','', text)
            if(text != ''):
                words.append(text)
        
        tag_words = nltk.pos_tag(words)
        pos_words = [word for word in tag_words if word[1][0] in {'V','N','J','R'}]
        # pos_words = [word if word[1].startswith('V') else word if word[1].startswith('N') else word if word[1].startswith('J') else word if word[1].startswith('R') else word for word in tag_words]
        temp_list = []
        for token, pos_tag in pos_words:
            tag = get_wordnet_pos(pos_tag)
            if tag != None:
                temp_list.append((token, get_wordnet_pos(pos_tag)))
        lemma = WordNetLemmatizer()
        token_final = [lemma.lemmatize(token, pos=tag) for token, tag in temp_list]
        long_words = [i for i in token_final if len(i) > 2]
        results = [w for w in long_words if w not in stop_words]
        return results

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

In [ ]:
custom_tokenizer = CustomTokenizer(text_to_word_sequence)
custom_tokenizer(sentence)

In [ ]:
tfidf = TfidfVectorizer(tokenizer=custom_tokenizer,ngram_range = (3, 3), stop_words=stop_words, max_features = 10000).fit(drama)
candidates = tfidf.get_feature_names()

In [47]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')
doc_embedding = model.encode([sentence])
candidate_embeddings = model.encode(candidates)

In [48]:
doc_embedding.shape

(1, 768)

In [49]:
from sklearn.metrics.pairwise import cosine_similarity

top_n = 5
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]

In [50]:
keywords

['masterful story little',
 'dark romantic motivate',
 'drama simple yet',
 'drama good plot',
 'half drama pretty']

mmr(중복 최소화)

In [51]:
import numpy as np

def mmr(doc_embedding, word_embeddings, words, top_n, diversity):

    # Extract similarity within words, and between words and the document
    word_doc_similarity = cosine_similarity(word_embeddings, doc_embedding)
    word_similarity = cosine_similarity(word_embeddings)

    # Initialize candidates and already choose best keyword/keyphras
    keywords_idx = [np.argmax(word_doc_similarity)]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    for _ in range(top_n - 1):
        # Extract similarities within candidates and
        # between candidates and selected keywords/phrases
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # Calculate MMR
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # Update keywords & candidates
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [words[idx] for idx in keywords_idx]

In [52]:
keywords

['masterful story little',
 'dark romantic motivate',
 'drama simple yet',
 'drama good plot',
 'half drama pretty']

In [53]:
mmr_drama = mmr(doc_embedding,candidate_embeddings,candidates,top_n=1000, diversity=0.7)

ngram_range = (3, 3)인 경우

In [54]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(mmr_drama)

true_k = 3
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(X)

print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(true_k):
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind],end = '')
    print("\n")

print("\n")
print("Prediction")

# Y = vectorizer.transform(["character"])
# prediction = model.predict(Y)
# print(prediction)

# Y = vectorizer.transform(["story"])
# prediction = model.predict(Y)
# print(prediction)

Top terms per cluster:
Cluster 0:
 story character life watch love plot make really episode people

Cluster 1:
 female wish lead savor annoy fairytale fulfillment best character food

Cluster 2:
 drama watch love usual historical life new recommend character hometown



Prediction


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


ngram_range = (1, 1)인 경우

In [45]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(mmr_drama)

true_k = 3
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(X)

print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(true_k):
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind],end = '')
    print("\n")


print("\n")
print("Prediction")

# Y = vectorizer.transform(["character"])
# prediction = model.predict(Y)
# print(prediction)

# Y = vectorizer.transform(["story"])
# prediction = model.predict(Y)
# print(prediction)

Top terms per cluster:
Cluster 0:
 zombie idealistic int instrumentals insanity influence indonesian individualistic indie incorporate

Cluster 1:
 domino zombie hyjein insanity influence indonesian individualistic indie incorporate inbetween

Cluster 2:
 tolstoy zombie hurdle insanity influence indonesian individualistic indie incorporate inbetween



Prediction


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
